In [19]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple, List
import os

In [20]:
@f.udf(returnType=t.FloatType())
def converter_minutos(tempo: str) -> float:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_segundos = duracao.total_seconds() 
    total_minutos = total_segundos / 60
   
    return round(total_minutos, 2)


In [21]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

In [22]:
def criar_particao_ouro(dataframe: DataFrame, nome_arquivo: str ):
    caminho_datalake_ouro = '../dados/ouro'
    dataframe.write. \
                parquet(os.path.join(caminho_datalake_ouro, nome_arquivo),  )

In [23]:
def transformacao_dataframe_trends(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        'DATA_PUBLICACAO',
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd" ))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_MINUTOS', converter_minutos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    dataframe_transform = dataframe_transform.withColumn('HORA_EXTRACAO', f.hour("data_hora_extracao"))
    dataframe_transform = dataframe_transform.withColumn(
        'TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.HORA_EXTRACAO >=6) & (dataframe_transform.HORA_EXTRACAO < 12), 'Manhã')
            .when((dataframe_transform.HORA_EXTRACAO >= 12) & (dataframe_transform.HORA_EXTRACAO < 18), 'Tarde')
            .otherwise('Noite')
        )
    )
    dataframe_transform = dataframe_transform.withColumn(
        'INDICE_TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.TURNO_EXTRACAO == 'Manhã'), 1)
            .when((dataframe_transform.TURNO_EXTRACAO == 'Tarde'), 2)
            .otherwise(3)
        )
    )
    return dataframe_transform
  

In [24]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'ASSUNTO',
        'data_extracao',
        'DATA_PUBLICACAO',
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd" ))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_MINUTOS', converter_minutos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    dataframe_transform = dataframe_transform.withColumn('HORA_EXTRACAO', f.hour("data_hora_extracao"))
    dataframe_transform = dataframe_transform.withColumn(
        'TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.HORA_EXTRACAO >=6) & (dataframe_transform.HORA_EXTRACAO < 12), 'Manhã')
            .when((dataframe_transform.HORA_EXTRACAO >= 12) & (dataframe_transform.HORA_EXTRACAO < 18), 'Tarde')
            .otherwise('Noite')
        )
    )
    dataframe_transform = dataframe_transform.withColumn(
        'INDICE_TURNO_EXTRACAO',
        (
            f.when(
                (dataframe_transform.TURNO_EXTRACAO == 'Manhã'), 1)
            .when((dataframe_transform.TURNO_EXTRACAO == 'Tarde'), 2)
            .otherwise(3)
        )
    )
    return dataframe_transform
  

In [25]:
@f.udf(returnType=t.StringType())
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [26]:
@f.udf(returnType=t.IntegerType())
def indice_semana(dia):
    traducao = {
        'Domingo': 1,
        'Segunda-feira': 2,
        'Terça-feira': 3,
        'Quarta-feira': 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado': 7
    }
    return traducao.get(dia, dia)

In [27]:
df_estatisticas_gerais = spark.read.parquet('../dados/prata/arquivos_assuntos.parquet')
df_estatisticas_gerais.columns

['data_extracao',
 'ID_VIDEO',
 'DATA_PUBLICACAO',
 'ID_CANAL',
 'NM_CANAL',
 'ID_CATEGORIA',
 'TITULO_VIDEO',
 'DESCRICAO',
 'TAGS',
 'DURACAO_VIDEOS',
 'TOTAL_VISUALIZACOES',
 'TOTAL_LIKES',
 'TOTAL_FAVORITOS',
 'TOTAL_COMENTARIOS',
 'ASSUNTO']

In [28]:
df_estatisticas_gerais = transformacao_dataframe(dataframe_transform=df_estatisticas_gerais)
df_estatisticas_gerais.show()

+--------------------+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+---------------------+---------------------+----------+-------------+--------------+---------------------+
|             ASSUNTO|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_MINUTOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|HORA_EXTRACAO|TURNO_EXTRACAO|INDICE_TURNO_EXTRACAO|
+--------------------+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+--------

In [29]:
def construir_depara_canal(dataframe: DataFrame):
    dataframe.select(
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO
    ).distinct().sort('ID_CANAL').write.\
        parquet(f'../dados/ouro/depara_canal.parquet')


In [30]:
def obter_total_publicacao_video(dataframe : DataFrame) -> DataFrame:
   dataframe =  dataframe.select(
    'DATA_PUBLICACAO',
    'ID_CANAL',
    'NM_CANAL'
    ).withColumn(
       'DATA_PUBLICACAO', f.date_format('DATA_PUBLICACAO', 'yyyy-MM-dd')
    ) \
        .sort('ID_CANAL') \
        .withColumn(
            'SEMANA_TRADUZIDA', 
            traduzir_dia_da_semana(
                f.date_format('DATA_PUBLICACAO', 'EEEE')
            )
            ).distinct() \
                .groupBy('DATA_PUBLICACAO','SEMANA_TRADUZIDA', 'ID_CANAL', 'NM_CANAL'). \
                    count(). \
                        alias('TOTAL_VIDEOS'). \
                            withColumn('INDICE_SEMANA', indice_semana('SEMANA_TRADUZIDA')) \
                    .sort('INDICE_SEMANA')\
                        .select('DATA_PUBLICACAO','SEMANA_TRADUZIDA' ,'ID_CANAL', 'NM_CANAL', f.col('TOTAL_VIDEOS.COUNT').alias('TOTAL_VIDEOS'))       
   return dataframe

In [31]:
df_estatisticas_gerais.show()

+--------------------+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+---------------------+---------------------+----------+-------------+--------------+---------------------+
|             ASSUNTO|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_MINUTOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|HORA_EXTRACAO|TURNO_EXTRACAO|INDICE_TURNO_EXTRACAO|
+--------------------+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+--------

In [32]:
df_teste = df_estatisticas_gerais.select(
    'ASSUNTO',
    'data_extracao',
    'ID_CANAL',
    'NM_CANAL',
    'ID_VIDEO',
    'TITULO_VIDEO',
    'TOTAL_VISUALIZACOES'
).orderBy([ 'data_extracao', 'ID_VIDEO'])

In [33]:
df_teste.show(truncate=False)

+-------------------------+-------------+------------------------+--------------+-----------+-------------------------------------------------------------------------------------+-------------------+
|ASSUNTO                  |data_extracao|ID_CANAL                |NM_CANAL      |ID_VIDEO   |TITULO_VIDEO                                                                         |TOTAL_VISUALIZACOES|
+-------------------------+-------------+------------------------+--------------+-----------+-------------------------------------------------------------------------------------+-------------------+
|assunto_genshin_impact   |2024-01-17   |UCd8uT2TYeZqeV9hJ5jMWBGA|Snukle        |01FD9IqMfq4|GENSHIN IMPACT EM 2024! #genshinimpact #shorts                                       |600                |
|assunto_genshin_impact   |2024-01-17   |UCd8uT2TYeZqeV9hJ5jMWBGA|Snukle        |01FD9IqMfq4|GENSHIN IMPACT EM 2024! #genshinimpact #shorts                                       |585                |


In [34]:
colunas_teste = (1, 2)
print(colunas_teste, *colunas_teste)

(1, 2) 1 2


In [35]:
df_teste.filter('ID_VIDEO == "-5JqqihLCE8"').show() 

+--------------------+-------------+--------------------+--------------------+-----------+------------+-------------------+
|             ASSUNTO|data_extracao|            ID_CANAL|            NM_CANAL|   ID_VIDEO|TITULO_VIDEO|TOTAL_VISUALIZACOES|
+--------------------+-------------+--------------------+--------------------+-----------+------------+-------------------+
|assunto_python_an...|   2024-01-23|UCHQun4keJGrj8RqD...|Portal do Conheci...|-5JqqihLCE8|xeque-mate ☠|                 60|
+--------------------+-------------+--------------------+--------------------+-----------+------------+-------------------+



In [36]:
df_estatisticas_gerais_pandas = df_estatisticas_gerais.toPandas()
df_estatisticas_gerais_pandas

,ASSUNTO,data_extracao,DATA_PUBLICACAO,ID_CANAL,NM_CANAL,ID_CATEGORIA,ID_VIDEO,TITULO_VIDEO,DESCRICAO,TAGS,...,TOTAL_LIKES,TOTAL_FAVORITOS,TOTAL_COMENTARIOS,data_hora_extracao,DURACAO_VIDEO_MINUTOS,TOTAL_CARACTERE_VIDEO,TOTAL_TAGS,HORA_EXTRACAO,TURNO_EXTRACAO,INDICE_TURNO_EXTRACAO
0,assunto_python_and_dados,2024-01-23,2024-01-19T17:35:48Z,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,27,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,"Diferença entre Dados, Informação, Conheciment...","[python, joao futi muanda, usando python, usan...",...,13.0,0,0.0,2024-01-23 10:37:43,0.570000,45,18,10,Manhã,1
1,assunto_python_and_dados,2024-01-23,2024-01-20T20:43:41Z,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,28,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024,"🚀 Bem-vindo ao nosso canal! Neste vídeo, vamos...",None,...,31.0,0,8.0,2024-01-23 17:08:13,4.780000,43,0,17,Tarde,2
2,assunto_python_and_dados,2024-01-23,2024-01-18T15:15:11Z,UCiYrw6iAqlYXvP7no7h4tUQ,Rodolfo De Nadai,28,s9hUoajuWwc,Python: Structure Data Types,Fala galera!\nNeste vídeo vou comentar das pri...,"[tecnologia, programming]",...,2.0,0,0.0,2024-01-23 10:37:41,26.730000,28,2,10,Manhã,1
3,assunto_python_and_dados,2024-01-23,2024-01-23T12:59:19Z,UCNblNh-u0nu5mtXT84Wgdiw,Mestres da Informática,27,j1-IbFM7TtE,🔴O Novo Ciclo - 1° Fase | O Detetive Virtual,Nessa Aula eu vou te mostrar como você pode co...,[detetive virtual],...,102.0,0,2.0,2024-01-23 20:00:58,110.000000,44,1,20,Noite,3
4,assunto_python_and_dados,2024-01-23,2024-01-18T22:00:02Z,UCU5vSFBBO_cp6ie0OG74b2g,Refatorando,28,5Rl2CnW5ZFc,Como é o dia a dia de trabalho de um Programador,→ 🎁 Masterclass Gratuita - Como Iniciar na Car...,"[roger santos, refatorando, Como é o dia a dia...",...,136.0,0,12.0,2024-01-23 20:00:55,9.980000,48,28,20,Noite,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12063,assunto_cities_skylines_2,2024-01-17,2024-01-17T13:52:45Z,UCGved3ynk6ZhUvSKxzgweTw,Jogando,20,tjZZTkpmPo4,"07 Itaquera - Cities Skylines 2 UNIVERSIDADE, ...",FAAAALA MEU POVO!\nBem vindos a mais um episód...,"[Canal Jogando, Canal Jogando Raiz, Cities, Ci...",...,0.0,0,0.0,2024-01-17 11:02:06,57.119999,85,25,11,Manhã,1
12064,assunto_cities_skylines_2,2024-01-17,2024-01-17T15:00:22Z,UCdrd9d0-1p8sgXOvJDc-naA,FLAVIOSUICO GAMES,20,J0O_HqOo4UE,Cities Skyline 2 - INTERSECÇÃO ENORME - Gamepl...,Jogo Cities Skylines 2 - 75K\n\nhttps://www.tw...,"[#Citiesskyline2, #cities, #skyline, #gameplay...",...,0.0,0,0.0,2024-01-17 17:01:08,88.750000,70,5,17,Tarde,2
12065,assunto_cities_skylines_2,2024-01-17,2024-01-18T00:30:06Z,UCDqzBrUOm7guqqyU8eDackg,BUG CITIES,20,AHKQdCqYoHI,Cidade (Mods) CITIES SKYLINES I (iniciante)| C...,(!!ESSE VIDEO TERÁ PARTES COM STUTERING!!)\n(!...,"[Simulação de cidade, Construção de cidade, Ge...",...,0.0,0,0.0,2024-01-17 21:51:26,10.100000,99,23,21,Noite,3
12066,assunto_cities_skylines_2,2024-01-17,2024-01-18T00:30:06Z,UCDqzBrUOm7guqqyU8eDackg,BUG CITIES,20,AHKQdCqYoHI,Cidade (Mods) CITIES SKYLINES I (iniciante)| C...,(!!ESSE VIDEO TERÁ PARTES COM STUTERING!!)\n(!...,"[Simulação de cidade, Construção de cidade, Ge...",...,0.0,0,0.0,2024-01-17 21:57:29,10.100000,99,23,21,Noite,3


In [37]:
for chave, coluna in enumerate(df_estatisticas_gerais_pandas.columns):
    try:
        print(chave , coluna)
        print(df_estatisticas_gerais_pandas[coluna].unique())
        print()
    except:
        print('erro na coluna' ,coluna)

0 ASSUNTO
['assunto_python_and_dados' 'assunto_power_bi' 'assunto_genshin_impact'
 'assunto_linux' 'assunto_linux_gamming' 'assunto_zelda'
 'assunto_cities_skylines' 'assunto_cities_skylines_2']

1 data_extracao
['2024-01-23' '2024-01-22' '2024-01-21' '2024-01-20' '2024-01-19'
 '2024-01-18' '2024-01-17']

2 DATA_PUBLICACAO
['2024-01-19T17:35:48Z' '2024-01-20T20:43:41Z' '2024-01-18T15:15:11Z' ...
 '2024-01-18T18:23:57Z' '2024-01-17T22:04:25Z' '2024-01-17T14:05:59Z']

3 ID_CANAL
['UCMqGy4xIIGs01ZVcBv0B8Cw' 'UC70kawNmZCDeZDZBbeiolHA'
 'UCiYrw6iAqlYXvP7no7h4tUQ' 'UCNblNh-u0nu5mtXT84Wgdiw'
 'UCU5vSFBBO_cp6ie0OG74b2g' 'UCHQun4keJGrj8RqDPPpyUsg'
 'UC_Ytk3TwR-bWEHFhHsKcVHw' 'UCUUFXfP7yc1pGC9h3oASYyQ'
 'UCIQ_CeDhXZDc8BEaMFFXi0g' 'UC70mr11REaCqgKke7DPJoLg'
 'UC2vF0_RY5Bqk-tMUJDokHNA' 'UCKJ2JOMmbgpoRnu7G6-notQ'
 'UCkfyKm2S5eSurJDt3TATq-A' 'UCyhhfAYg6psBJyNOUwE-nwA'
 'UCDqfUwybgEA9Hg3P32G4Uaw' 'UC_1o3kVaVVMDrWvAbrdmFkQ'
 'UCar5Cr-pVz08GY_6I3RX9bA' 'UCKlK9CmpSwvvYaeXB9ciqdg'
 'UCTEVZSr6cu4LPtcfj1Nb

In [38]:
df_estatisticas_gerais_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12068 entries, 0 to 12067
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ASSUNTO                12068 non-null  object 
 1   data_extracao          12068 non-null  object 
 2   DATA_PUBLICACAO        12068 non-null  object 
 3   ID_CANAL               12068 non-null  object 
 4   NM_CANAL               12068 non-null  object 
 5   ID_CATEGORIA           12068 non-null  object 
 6   ID_VIDEO               12068 non-null  object 
 7   TITULO_VIDEO           12068 non-null  object 
 8   DESCRICAO              12068 non-null  object 
 9   TAGS                   8647 non-null   object 
 10  DURACAO_VIDEOS         12068 non-null  object 
 11  TOTAL_VISUALIZACOES    12065 non-null  float64
 12  TOTAL_LIKES            11842 non-null  float64
 13  TOTAL_FAVORITOS        12068 non-null  int32  
 14  TOTAL_COMENTARIOS      12050 non-null  float64
 15  da

In [39]:
for chave, coluna in enumerate(df_estatisticas_gerais_pandas.columns):
    if chave in [0, 3, 4, 6, 10, 20, 7, 8]:
        df_estatisticas_gerais_pandas[coluna] = df_estatisticas_gerais_pandas[coluna].astype('string')
    elif chave in [1, 2, 5]:
        df_estatisticas_gerais_pandas[coluna] = df_estatisticas_gerais_pandas[coluna].astype('string')
    elif chave in [5, 11, 12, 13, 14, 17, 18, 19, 21]:
        df_estatisticas_gerais_pandas[coluna] = df_estatisticas_gerais_pandas[coluna].fillna(0).astype('int32')

In [40]:
df_estatisticas_gerais_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12068 entries, 0 to 12067
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ASSUNTO                12068 non-null  string 
 1   data_extracao          12068 non-null  string 
 2   DATA_PUBLICACAO        12068 non-null  string 
 3   ID_CANAL               12068 non-null  string 
 4   NM_CANAL               12068 non-null  string 
 5   ID_CATEGORIA           12068 non-null  string 
 6   ID_VIDEO               12068 non-null  string 
 7   TITULO_VIDEO           12068 non-null  string 
 8   DESCRICAO              12068 non-null  string 
 9   TAGS                   8647 non-null   object 
 10  DURACAO_VIDEOS         12068 non-null  string 
 11  TOTAL_VISUALIZACOES    12068 non-null  int32  
 12  TOTAL_LIKES            12068 non-null  int32  
 13  TOTAL_FAVORITOS        12068 non-null  int32  
 14  TOTAL_COMENTARIOS      12068 non-null  int32  
 15  da

In [41]:
df_estatisticas_gerais_pandas.to_parquet('../dados/ouro/dados_tratado_estatisticas_gerais.parquet')

In [19]:
criar_particao_ouro(
        dataframe=df_estatisticas_gerais, 
        nome_arquivo='dados_tratado_estatisticas_gerais.parquet',
    )

24/02/02 21:32:14 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers
24/02/02 21:32:14 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 84,47% for 8 writers
24/02/02 21:32:15 WARN MemoryManager: Total allocation exceeds 95,00% (906.992.014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers


In [25]:
df_trends = spark.read.parquet('/home/rodrigo/Documentos/projetos/analise_dados_youtube/dados/prata/arquivos_assuntos_trends.parquet')
df_trends = transformacao_dataframe_trends(df_trends)
criar_particao_ouro(dataframe=df_trends, nome_arquivo='estatisticas_trends.parquet')
df_trends.show()

+-------------+--------------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+---------------------+---------------------+----------+-------------+--------------+---------------------+
|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|          NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_MINUTOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|HORA_EXTRACAO|TURNO_EXTRACAO|INDICE_TURNO_EXTRACAO|
+-------------+--------------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-----------

In [79]:
spark.stop()